# **Calculation of the average covariance matrix of productivities**


### Connecting to Google Drive to see the data

In [ ]:
from google.colab import drive
drive.mount ("/content/drive")

Mounted at /content/drive


### Loading productivity data

In [ ]:
import pandas as pd

final_data = pd.read_csv('/content/drive/MyDrive/Data Competitions/BCG Gamma 2022/Dados - GAMMA Challenge 2022/Dados Finais/DadosFinais.csv')

In [ ]:
final_data

,Produtividade,Ano,Produto,CD_MUN,NM_REGIAO,NM_UF,CD_UF,SIGLA,NM_MUN,AREA_TOT,...,"PRECIPITACAO TOTAL, MENSAL (AUT)(mm)Ano Atual","PRECIPITACAO TOTAL, MENSAL (AUT)(mm)Ano Passado","PRESSAO ATMOSFERICA, MEDIA MENSAL (AUT)(mB)Ano Atual","PRESSAO ATMOSFERICA, MEDIA MENSAL (AUT)(mB)Ano Passado","TEMPERATURA MEDIA, MENSAL (AUT)(°C)Ano Atual","TEMPERATURA MEDIA, MENSAL (AUT)(°C)Ano Passado","VENTO, VELOCIDADE MAXIMA MENSAL (AUT)(m/s)Ano Atual","VENTO, VELOCIDADE MAXIMA MENSAL (AUT)(m/s)Ano Passado","VENTO, VELOCIDADE MEDIA MENSAL (AUT)(m/s)Ano Atual","VENTO, VELOCIDADE MEDIA MENSAL (AUT)(m/s)Ano Passado"
0,16.818182,2010,Abacaxi,1100015,NORTE,Rondônia,11,RO,Alta Floresta D'Oeste,7067.13,...,0.000,0.500000,977.687899,976.771272,25.055975,24.759350,NaN,5.916667,NaN,1.331133
1,18.838710,2010,Mandioca,1100015,NORTE,Rondônia,11,RO,Alta Floresta D'Oeste,7067.13,...,0.000,0.500000,977.687899,976.771272,25.055975,24.759350,NaN,5.916667,NaN,1.331133
2,14.600000,2010,Melancia,1100015,NORTE,Rondônia,11,RO,Alta Floresta D'Oeste,7067.13,...,0.000,0.500000,977.687899,976.771272,25.055975,24.759350,NaN,5.916667,NaN,1.331133
3,0.576471,2010,Cacau (em Amendoa),1100015,NORTE,Rondônia,11,RO,Alta Floresta D'Oeste,7067.13,...,0.000,0.500000,977.687899,976.771272,25.055975,24.759350,NaN,5.916667,NaN,1.331133
4,24.333333,2010,Mamao,1100015,NORTE,Rondônia,11,RO,Alta Floresta D'Oeste,7067.13,...,0.000,0.500000,977.687899,976.771272,25.055975,24.759350,NaN,5.916667,NaN,1.331133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,22.533333,2019,Melancia,5107958,CENTRO-OESTE,Mato Grosso,51,MT,Tangará da Serra,11636.98,...,87.550,90.828571,962.703557,962.928285,25.045269,24.027463,8.466667,8.442857,2.360606,2.459531
25719,1.500000,2019,Cafe (em Grao) Canephora,5107958,CENTRO-OESTE,Mato Grosso,51,MT,Tangará da Serra,11636.98,...,87.550,90.828571,962.703557,962.928285,25.045269,24.027463,8.466667,8.442857,2.360606,2.459531
25720,15.000000,2019,Maracuja,5107958,CENTRO-OESTE,Mato Grosso,51,MT,Tangará da Serra,11636.98,...,87.550,90.828571,962.703557,962.928285,25.045269,24.027463,8.466667,8.442857,2.360606,2.459531
25721,12.000000,2019,Mandioca,5108352,CENTRO-OESTE,Mato Grosso,51,MT,Vale de São Domingos,1901.55,...,29.175,30.500000,982.425064,982.735916,25.352462,25.000591,6.590000,6.383333,2.344532,2.396275


### **Calculation of the desired matrix**

In [ ]:
def fix_df(df, product_list):
  
    # finding out which products are not present in the given city
    missing_products = [i for i in product_list if i not in df.columns]

    # initializing the variable
    standard_df = df

    # adding missing columns
    standard_df[missing_products] = 0

    # adding the missing columns
    df_add = pd.DataFrame(len(missing_products)*[[0]*len(standard_df.columns)], index=missing_products, columns=standard_df.columns)
    standard_df = standard_df.fillna(0)
    standard_df = pd.concat([standard_df, df_add])

    # ordering columns and indexes
    standard_df = standard_df.sort_index()
    standard_df = standard_df.sort_index(1)

    return standard_df

Let's use the above function to replace with zero the columns and rows excluded by the .cov() function. This is necessary because the possible products per municipality are not the same.

In [ ]:
final_data = final_data[['Produto', 'Produtividade','Ano','NM_MUN','CD_MUN']]

# list with ids of the cities
city_list = final_data['CD_MUN'].unique()

# list of selected products
product_list = final_data['Produto'].unique()

# initializing variables
sum_covariance_matrix = None
weights = None
first = True

# analyzing the data of each city
for city in city_list:
  
    # filtering the city
    city_data = final_data.loc[final_data['CD_MUN'] == city]
    city_data = city_data[['Produto','Produtividade', 'Ano', 'NM_MUN']]

    # Rearranging the data into the desired format for calculating the covariance
    city_data = city_data.pivot(index='Ano', columns='Produto', values='Produtividade')
    city_data.columns.name = ''

    if first:

        # calculating the covariance matrix of the city products
        sum_covariance_matrix = city_data.cov()

        # adding products that do not exist in the city to standardize the matrix for all cities
        sum_covariance_matrix = fix_df(sum_covariance_matrix, product_list)

        # primeiros valores da matriz de pesos que serão utilizados para cálculo de uma média de covariância para cada produto
        weights = (sum_covariance_matrix != 0)*1
        first = False

    else:

        # calculating the covariance matrix to be added
        new_covariance_matrix = city_data.cov()

        # adding products that do not exist in the city to standardize the matrix for all cities
        new_covariance_matrix = fix_df(new_covariance_matrix, product_list)

        # adding to the others
        sum_covariance_matrix = sum_covariance_matrix.add(new_covariance_matrix)

        # adding the weights to the weight matrix
        weights = weights.add((new_covariance_matrix != 0)*1)

# calculate the average term by term
average_covariance_matrix = sum_covariance_matrix.divide(weights)
average_covariance_matrix

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:9524: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


,Abacaxi,Acai,Cacau (em Amendoa),Cafe (em Grao) Canephora,Castanha de Caju,Mamao,Mandioca,Maracuja,Melancia,Pimenta do Reino,Urucum (Semente)
Abacaxi,15.867857,-0.798444,0.084078,0.191693,-0.042885,1.145247,1.121729,1.775079,2.631686,0.038103,0.196762
Acai,-0.798444,4.073473,0.008047,0.186218,-0.113403,-1.052097,0.136713,-0.662358,-0.969769,0.001797,0.023153
Cacau (em Amendoa),0.084078,0.008047,0.258679,0.001814,-0.003322,0.005451,-0.040642,-0.019190,-0.038092,0.008672,-0.016301
Cafe (em Grao) Canephora,0.191693,0.186218,0.001814,0.209419,0.031419,-1.056381,0.659068,-0.567083,0.744823,0.124466,-0.008442
Castanha de Caju,-0.042885,-0.113403,-0.003322,0.031419,0.066342,-0.034254,0.032802,0.076410,0.023329,-0.055967,0.003862
Mamao,1.145247,-1.052097,0.005451,-1.056381,-0.034254,30.135295,-0.990045,5.185585,-1.271012,-0.084009,0.228318
Mandioca,1.121729,0.136713,-0.040642,0.659068,0.032802,-0.990045,8.953633,-0.516083,2.007928,0.159789,-0.341655
Maracuja,1.775079,-0.662358,-0.019190,-0.567083,0.076410,5.185585,-0.516083,12.032885,0.113771,-0.001695,0.057595
Melancia,2.631686,-0.969769,-0.038092,0.744823,0.023329,-1.271012,2.007928,0.113771,21.156034,0.085569,0.369554
Pimenta do Reino,0.038103,0.001797,0.008672,0.124466,-0.055967,-0.084009,0.159789,-0.001695,0.085569,0.300030,0.036520


In [ ]:
average_covariance_matrix.to_csv('/content/drive/MyDrive/Data Competitions/BCG Gamma 2022/Dados - GAMMA Challenge 2022/Dados Finais/MatrizCovariancia.csv')